## Jetson Training

The following notebook will be used to help the user set up training using Google Colab. 

## Download Necessary Packages

In [ ]:
!apt-get update
!apt-get install git cmake libpython3-dev python3-numpy
!git clone --recursive https://github.com/dusty-nv/jetson-inference
!cd jetson-inference
!mkdir build
!cd build
!cmake ../
!make -j$(nproc)
!sudo make install
!sudo ldconfig

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,392 kB]
G

## Upload the images and annotations by doing the following steps.

First, upload TrainingTools.zip by opening up the folder on the left and clicking the file with arrow pointing up. Then, click the cell below to unzip.

In [ ]:
!unzip TrainingTools

Archive:  TrainingTools.zip
  inflating: TrainingTools/README.md  
   creating: TrainingTools/ssd/
  inflating: TrainingTools/ssd/.gitignore  
  inflating: TrainingTools/ssd/onnx_export.py  
  inflating: TrainingTools/ssd/readImage.py  
 extracting: TrainingTools/ssd/requirements.txt  
  inflating: TrainingTools/ssd/train_ssd.py  
  inflating: TrainingTools/ssd/ValidName.py  
   creating: TrainingTools/ssd/vision/
 extracting: TrainingTools/ssd/vision/__init__.py  
   creating: TrainingTools/ssd/vision/datasets/
 extracting: TrainingTools/ssd/vision/datasets/__init__.py  
  inflating: TrainingTools/ssd/vision/datasets/collation.py  
  inflating: TrainingTools/ssd/vision/datasets/generate_vocdata.py  
  inflating: TrainingTools/ssd/vision/datasets/open_images.py  
  inflating: TrainingTools/ssd/vision/datasets/voc_dataset.py  
   creating: TrainingTools/ssd/vision/nn/
 extracting: TrainingTools/ssd/vision/nn/__init__.py  
  inflating: TrainingTools/ssd/vision/nn/alexnet.py  
  inflating

Then, we need to upload two files, Annotations and JPEGImages onto the Google Drive in their own respective folders. The bottom cell will mount your drive and its content to the notebook to use.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Move Annotations and JPEGImages to TrainingTools/ssd/

Move the folders to the SSD Directory

In [ ]:
!mv drive/MyDrive/Annotations/ TrainingTools/ssd/
!mv drive/MyDrive/JPEGImages/ TrainingTools/ssd/

The bottom cell creates the ImageSets/Main directory and creates the train.txt and trainval.txt files and moves it to the ImageSets/Main directory.

In [ ]:

%cd TrainingTools/ssd
!mkdir ImageSets
!mkdir ImageSets/Main
!python3 readImage.py
!mv train.txt ImageSets/Main/
%cd ImageSets/Main/
!cp train.txt trainval.txt
%cd ../..
!pwd

/content/TrainingTools/ssd
/content/TrainingTools/ssd/ImageSets/Main
/content/TrainingTools/ssd
/content/TrainingTools/ssd


In [ ]:
# Upload labels.txt before running
%cd /content/
!mv labels.txt TrainingTools/ssd
%cd TrainingTools/ssd

/content
mv: cannot stat 'labels.txt': No such file or directory
/content/TrainingTools/ssd


Before running the cell below, upload a labels.txt file which will be used for training

In [ ]:
!pwd
!python3 vision/datasets/generate_vocdata.py ./labels.txt

/content/TrainingTools/ssd
Labels: ['Stage1.2'] imgcnt: 992
Missing annotation for  Annotations/20210218_165229 (1).xml


Bottom cell creates a models/Capstone directory and install requirements

In [ ]:
!mkdir models
!mkdir models/Capstone
!wget https://nvidia.box.com/shared/static/djf5w54rjvpqocsiztzaandq1m3avr7c.pth -O models/mobilenet-v1-ssd-mp-0_675.pth
!pip3 install -v -r requirements.txt

--2021-02-28 16:21:41--  https://nvidia.box.com/shared/static/djf5w54rjvpqocsiztzaandq1m3avr7c.pth
Resolving nvidia.box.com (nvidia.box.com)... 107.152.26.197
Connecting to nvidia.box.com (nvidia.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/djf5w54rjvpqocsiztzaandq1m3avr7c.pth [following]
--2021-02-28 16:21:42--  https://nvidia.box.com/public/static/djf5w54rjvpqocsiztzaandq1m3avr7c.pth
Reusing existing connection to nvidia.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://nvidia.app.box.com/public/static/djf5w54rjvpqocsiztzaandq1m3avr7c.pth [following]
--2021-02-28 16:21:42--  https://nvidia.app.box.com/public/static/djf5w54rjvpqocsiztzaandq1m3avr7c.pth
Resolving nvidia.app.box.com (nvidia.app.box.com)... 107.152.26.201
Connecting to nvidia.app.box.com (nvidia.app.box.com)|107.152.26.201|:443... connected.
HTTP request sent, awaiting response... 302 Found
Loca

Bottom begins training

In [ ]:
!python3 train_ssd.py --dataset-type=voc --data=./ --model-dir=models/Capstone --batch-size=4 --workers=2 --epochs=10

## Once everything is complete, remember to download the models/Capstone folder to your local machine. If the runtime ends or window or tab is closed, all data will be lost so make sure to download it. Since we need CUDA cores to make the .onnx conversion, the conversion needs to be done on the Jetson.